In [5]:
pip install pandas openpyxl nltk textblob pyphen


     ---------------------------------------- 2.0/2.0 MB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob
import pyphen

# Load the extracted article texts
input_directory = "extracted_articles"  # Directory where the extracted article text files are stored
output_structure_file = "Output Data Structure.xlsx"
output_file = "output.xlsx"

# Initialize the hyphenation dictionary for syllable count
dic = pyphen.Pyphen(lang='en_US')

# Function to compute textual analysis variables
def compute_variables(article_text):
    # Tokenize sentences and words
    sentences = sent_tokenize(article_text)
    words = word_tokenize(article_text)
    
    # Calculate variables based on TextBlob analysis
    text_blob = TextBlob(article_text)
    polarity_score = text_blob.sentiment.polarity
    subjectivity_score = text_blob.sentiment.subjectivity
    
    # Calculate additional variables
    avg_sentence_length = len(words) / len(sentences)
    word_count = len(words)
    complex_word_count = len([word for word in words if len(word) >= 7])
    percentage_complex_words = (complex_word_count / word_count) * 100 if word_count > 0 else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = word_count / len(sentences)

    # Calculate syllable count
    syllable_count = sum([len(dic.inserted(word).split("-")) for word in words])
    syllables_per_word = syllable_count / word_count if word_count > 0 else 0

    # Calculate personal pronouns count
    personal_pronouns = ["I", "me", "my", "mine"]
    personal_pronoun_count = sum([words.count(pp) for pp in personal_pronouns])

    # Calculate average word length
    avg_word_length = sum([len(word) for word in words]) / word_count if word_count > 0 else 0

    return (
        polarity_score,
        1 - polarity_score,  # Negative Score
        polarity_score,  # Polarity Score
        subjectivity_score,
        avg_sentence_length,
        percentage_complex_words,
        fog_index,
        avg_words_per_sentence,
        complex_word_count,
        word_count,
        syllables_per_word,
        personal_pronoun_count,
        avg_word_length
    )

# Read the output structure Excel file to get the variable names
output_structure_df = pd.read_excel(output_structure_file)

# Create a list of DataFrames to store the computed variables
dfs = []

# Iterate through the article text files
for filename in os.listdir(input_directory):
    if filename.endswith(".txt"):
        # Extract URL_ID from the filename (assuming the filename format is URL_ID.txt)
        url_id = os.path.splitext(filename)[0]

        # Read the article text from the file
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as text_file:
            article_text = text_file.read()

        # Compute variables for the article text
        computed_values = compute_variables(article_text)

        # Create a dictionary with the computed variables
        computed_variables = dict(zip(output_structure_df.columns, [url_id] + list(computed_values)))

        # Append the computed variables to the list of DataFrames
        dfs.append(pd.DataFrame([computed_variables]))

# Concatenate the DataFrames in the list to create the final DataFrame
computed_variables_df = pd.concat(dfs, ignore_index=True)

# Save the computed variables to an output Excel file
computed_variables_df.to_excel(output_file, index=False)

print(f"Output saved to {output_file}")


Output saved to output.xlsx
